# Pipline para datos.gob.ar

# Requerimientos funcionales 🔎
Tu proyecto deberá cumplir con una serie de requerimientos funcionales que giran
en torno a cuatro ejes centrales: los archivos fuente, el procesamiento de datos, la
creación de tablas en la base de datos y la actualización de la base de datos.
Veamos cada uno de ellos en detalle.

# Archivos fuente ✔
Los archivos fuentes serán utilizados en tu proyecto para obtener de ellos todo lo
necesario para popular la base de datos. El proyecto deberá:
- Obtener los 3 archivos de fuente utilizando la librería requests y
almacenarse en forma local (Ten en cuenta que las urls pueden cambiar en
un futuro):  
    - Datos Argentina - Museos
    - Datos Argentina - Salas de Cine
    - Datos Argentina - Bibliotecas Populares  
- Organizar los archivos en rutas siguiendo la siguiente estructura:
“categoría\año-mes\categoria-dia-mes-año.csv”
    - Por ejemplo: “museos\2021-noviembre\museos-03-11-2021”
    - Si el archivo existe debe reemplazarse. La fecha de la nomenclatura
es la fecha de descarga.

## Importaciones

In [21]:
# Modulos
import datetime
import locale
import requests
import pandas as pd
import os

## URLs

In [22]:
# Constantes
# urls
URL_MUSEOS = 'https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/4207def0-2ff7-41d5-9095-d42ae8207a5d/download/museos_datosabiertos.csv'
URL_CINES = 'https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/392ce1a8-ef11-4776-b280-6f1c7fae16ae/download/cine.csv'
URL_BIBLIOTECAS = 'https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/01c6c048-dbeb-44e0-8efa-6944f73715d7/download/biblioteca_popular.csv'


# Logs
El programa debe crear logs oportunos sobre la ejecución del mismo con la librería
Logging.

In [23]:
import logging

# Paths
mainpath = os.path.abspath(os.getcwd())
fullpath = mainpath + "/logging_info.log"

# Create and configure  a logger
LOG_FORMAT = "%(levelname)s %(asctime)s - %(message)s"
logging.basicConfig(filename = fullpath,
                    level = logging.DEBUG,
                    format = LOG_FORMAT,
                    filemode = 'w')
                   
logger = logging.getLogger()


## Escritura de archivo

In [24]:
# Peticion y carga de los ficheros
def requests_writing_reading(name, url):
    # variables
    #   time
    locale.setlocale(locale.LC_ALL, 'es-ES')
    dt = datetime.datetime.now()
    current_month = dt.strftime('%Y-%B')
    today = dt.strftime('%d-%m-%Y')
    #   paths
    main_path = os.path.abspath(os.getcwd())
    file_dir = fr'{name}\{current_month}'
    file_name = f'/{name}-{today}.csv'
    fullpath = os.path.join(main_path, file_dir + file_name)
    
    # making dir
    os.makedirs(file_dir, exist_ok= True)

    # requests
    r = requests.get(url)

    # Storage
    with open(fullpath, 'w', encoding="latin-1") as f:
        f.write(r.text.replace('\r',''))
    
    # Reading
    return pd.read_csv(fullpath)


In [25]:
# Dataframes
df_museos = requests_writing_reading('museos', URL_MUSEOS)
df_cines = requests_writing_reading('cines', URL_CINES)
df_bibliotecas = requests_writing_reading('bibliotecas', URL_BIBLIOTECAS)

In [26]:
# 🔍
df_museos.shape, df_cines.shape, df_bibliotecas.shape

((1182, 24), (329, 26), (2017, 25))

# Procesamiento de datos ✔
El procesamiento de datos permitirá a nuestro proyecto transformar los datos de los
archivos fuente en la información que va a nutrir la base de datos. Para esto, el
proyecto deberá:

- Normalizar toda la información de Museos, Salas de Cine y Bibliotecas
Populares, para crear una única tabla que contenga:
    - cod_localidad
    - id_provincia
    - id_departamento
    - categoría
    - provincia
    - localidad
    - nombre
    - domicilio
    - código postal
    - número de teléfono
    - mail
    - web
- Procesar los datos conjuntos para poder generar una tabla con la siguiente
información:
    - Cantidad de registros totales por categoría
    - Cantidad de registros totales por fuente
    - Cantidad de registros por provincia y categoría
- Procesar la información de cines para poder crear una tabla que contenga:
    - Provincia
    - Cantidad de pantallas
    - Cantidad de butacas
    - Cantidad de espacios INCAA

## Limpieza de las columnas solicitadas

In [27]:
# Para utilizar posteriormente
df_cines_copy = df_cines.copy()

In [28]:
# Unificando numero de telefono y codigo de area:
def concat_float_and_str_columns(df, float_column, str_column):
    """Funcion que concatena dos columnas en un dataframe, una correspondiente a tipo Str y otra a Float"""
    float_to_str = lambda x: str(x)[:-2]
    concat_columns = df[float_column].agg(float_to_str) + ' ' + df[str_column]
    return concat_columns.str.strip()

# Unificando numero de telefono y codigo de area:
df_museos.insert(13, 'telef.', concat_float_and_str_columns(df_museos, 'cod_area', 'telefono'))
df_cines.insert(13, 'telef.', concat_float_and_str_columns(df_cines, 'cod_area', 'Teléfono'))
df_bibliotecas.insert(13, 'telef.', concat_float_and_str_columns(df_bibliotecas, 'Cod_tel', 'Teléfono'))

# Drop columns
columns_to_drop_museos = ['cod_area','telefono','Observaciones', 'subcategoria','piso','Latitud','Longitud', 'TipoLatitudLongitud','Info_adicional']
columns_to_drop_cines = ['cod_area','Teléfono','Observaciones', 'Departamento','Piso','Información adicional','Latitud','Longitud','TipoLatitudLongitud']
columns_to_drop_bibliotecas = ['Cod_tel','Teléfono','Observacion', 'Subcategoria','Departamento','Piso','Información adicional','Latitud','Longitud', 'TipoLatitudLongitud']

# Eliminando las columnas innecesarías
df_museos = df_museos.drop(columns_to_drop_museos, axis=1)
df_cines = df_cines.drop(columns_to_drop_cines, axis=1)
df_bibliotecas = df_bibliotecas.drop(columns_to_drop_bibliotecas, axis=1)


## Chequeo de los datos

In [29]:
# Chequeo de que los campos coincidan en posicion
# 🔍
for i in range(13):
    print(i,df_museos.columns[i], df_cines.columns[i], df_bibliotecas.columns[i])

0 Cod_Loc Cod_Loc Cod_Loc
1 IdProvincia IdProvincia IdProvincia
2 IdDepartamento IdDepartamento IdDepartamento
3 categoria Categoría Categoría
4 provincia Provincia Provincia
5 localidad Localidad Localidad
6 nombre Nombre Nombre
7 direccion Dirección Domicilio
8 CP CP CP
9 telef. telef. telef.
10 Mail Mail Mail
11 Web Web Web
12 fuente Fuente Fuente


## Unificacion de los datasets

In [30]:
# Columnas necesarias
columns = ['cod_localidad',
          'id_provincia',
          'id_departamento',
          'categoria',
          'provincia',
          'localidad',
          'nombre',
          'domicilio',
          'codigo_postal',
          'telefono',
          'mail',
          'web',
          'fuente',]

# Concatenamiento en un solo dataset
df_main = pd.DataFrame()
for i, v in enumerate(columns):
    df_main[v] = pd.concat([df_museos.iloc[:,i], df_cines.iloc[:,i], df_bibliotecas.iloc[:,i]])

In [31]:
# 🔍
df_main.shape

(3528, 13)

## Generacion de los datasets agrupados

In [32]:
# Datasets agrupados
df_count_categoria = df_main.groupby('categoria').count()['cod_localidad']
df_count_fuente = df_main.groupby('fuente').count()['cod_localidad']
df_count_provincia_categoria = df_main.groupby(['provincia','categoria']).count()['cod_localidad']

In [33]:
# 🔍
print(df_count_categoria,
      "\n---------------------------------------------\n",
      df_count_fuente,
      "\n---------------------------------------------\n",
      df_count_provincia_categoria)

categoria
Bibliotecas Populares                 2017
Espacios de Exhibición Patrimonial    1182
Salas de cine                          329
Name: cod_localidad, dtype: int64 
---------------------------------------------
 fuente
CNMLH - Enlace SInCA                                                      1
CNMLH - Ente Cultural de Tucumán                                          1
CONABIP                                                                2010
DNPyM                                                                  1048
Dirección Provincial de Patrimonio Cultural - Enlace SInCA                1
Dirección de Cultura de Villa Allende                                     2
Dirección de Cultura y de Educación de la Municipalidad de Unquillo       1
Gob. Pcia.                                                                1
Gobierno de la Provincia                                                 14
Gobierno de la Provincia de Chubut / SInCA 2013                           3
Gobierno de 

## Tablas solicitadas

### ___Tabla 1___
- Normalizar toda la información de Museos, Salas de Cine y Bibliotecas
Populares, para crear una única tabla que contenga:
    - cod_localidad
    - id_provincia
    - id_departamento
    - categoría
    - provincia
    - localidad
    - nombre
    - domicilio
    - código postal
    - número de teléfono
    - mail
    - web

In [34]:
# Tabla solo con los campos especificados, (se excluje 'fuente')
df_info_1 = df_main.iloc[ : , :-1]

In [35]:
# 🔍
df_info_1.shape

(3528, 12)

### ___Tabla 2___

Procesar los datos conjuntos para poder generar una tabla con la siguiente
información:
- Cantidad de registros totales por categoría
- Cantidad de registros totales por fuente
- Cantidad de registros por provincia y categoría

In [36]:
# Campos requeridos de la tabla base
df_info_2 = df_main[['provincia', 'categoria', 'fuente']]

# Datos agrupados que seran agragados a la tabla
data_to_agregate = [(df_count_categoria, 'categoria', 'registros_categoria'),
                    (df_count_fuente, 'fuente', 'registros_fuente'),
                    (df_count_provincia_categoria, ['provincia', 'categoria'], 'registros_prov_cat'),]

# Agregacion de la informacion
for data in data_to_agregate:
    df_info_2 = pd.merge(left= df_info_2, right= data[0], how= 'left', on= data[1])
    df_info_2 = df_info_2.rename(columns= {'cod_localidad': data[2]})


In [37]:
# 🔍
df_info_2.iloc[[0,40,170,580,1560]]

,provincia,categoria,fuente,registros_categoria,registros_fuente,registros_prov_cat
0,Buenos Aires,Espacios de Exhibición Patrimonial,DNPyM,1182,1048,152
40,Buenos Aires,Espacios de Exhibición Patrimonial,DNPyM,1182,1048,152
170,Catamarca,Espacios de Exhibición Patrimonial,DNPyM,1182,1048,32
580,Entre Ríos,Espacios de Exhibición Patrimonial,DNPyM,1182,1048,57
1560,Buenos Aires,Bibliotecas Populares,CONABIP,2017,2010,543


### ___Tabla 3___
Procesar la información de cines para poder crear una tabla que contenga:
- Provincia
- Cantidad de pantallas
- Cantidad de butacas
- Cantidad de espacios INCAA

In [38]:
# Nueva lectura de cines
df_cines = df_cines_copy

# Convertir en valores numericos el recuento espacio_INCAA
df_cines['espacio_INCAA'] = df_cines['espacio_INCAA'].transform(lambda x: 1 if x == 'si' or x == 'SI' else 0)

# Creando la tabla agrupada por provincia
columns = ['Provincia', 'Pantallas', 'Butacas', 'espacio_INCAA']
df_info_3 = df_cines[columns].groupby('Provincia').sum()
df_info_3 = df_info_3.rename(columns= {'espacio_INCAA':'espacios_INCAA'})
df_info_3['Provincia'] = df_info_3.index

In [39]:
# 🔍
df_info_3.head()

,Pantallas,Butacas,espacios_INCAA,Provincia
Provincia,,,,
Buenos Aires,358,93112,20,Buenos Aires
Catamarca,12,3200,1,Catamarca
Chaco,14,2469,1,Chaco
Chubut,10,2682,4,Chubut
Ciudad Autónoma de Buenos Aires,153,31386,3,Ciudad Autónoma de Buenos Aires


# Creación de tablas / bbdd ✔

Para disponibilizar la información obtenida y procesada en los pasos previos, tu
proyecto deberá tener una base de datos que cumpla con los siguientes requisitos:
- La base de datos debe ser PostgreSQL/MySql
- Se deben crear los scripts .sql para la creación de las tablas.
- Se debe crear un script .py que ejecute los scripts .sql para facilitar el deploy.
- Los datos de la conexión deben poder configurarse fácilmente para facilitar
el deploy en un nuevo ambiente de ser necesario.

## Variables de entorno

In [40]:
from sqlalchemy import create_engine

# Variables de conexion
MOTOR = 'mysql'
USER = 'root'
PASSWORD = '1234'
HOST = '127.0.0.1'
PORT = '3306'
DATA_BASE = 'bbdd'

## Creacion de la base de datos

In [41]:
# Connection string for the engine
CONNECTION_STRING = MOTOR + '://' + USER + ':' + PASSWORD + '@' + HOST + ':' + PORT
# creacion del motor
engine = create_engine(CONNECTION_STRING)#, echo= True)

# Creacion de la base de datos y las tablas
with engine.connect() as con:
    with open('bbdd.sql', 'r') as f:
        con.execute(f.read())

# Actualización de la bbdd ✔
Luego de normalizar la información y generar las demás tablas, las mismas se
deben actualizar en la base de datos. Para eso, es importante tener en cuenta que:
- Todos los registros existentes deben ser reemplazados por la nueva
información.
- Dentro de cada tabla debe indicarse en una columna adicional la fecha de
carga.
- Los registros para los cuales la fuente no brinda información deben cargarse
como nulos.

In [42]:
# Se agrega la base de datos al engine
engine = create_engine(CONNECTION_STRING + '/' + DATA_BASE)

# Actualizado valores en la base de datos:
for i, df in enumerate([df_info_1, df_info_2, df_info_3]):
    # Borrado de los datos previos
    with engine.connect() as con:
        con.execute(f'DELETE FROM info_{i + 1}')
    # Carga de los nuevos datos
    df.to_sql(f'info_{i + 1}',con = engine, index = False, if_exists = 'append', method=  'multi')

# Requerimientos técnicos 🔧
Tu aplicación deberá cumplir con una serie de requerimientos técnicos que giran en
torno a 7 ejes centrales. Veamos cada uno de ellos en detalle.

# Deploy ✔
El proyecto debe poder deployarse en forma sencilla siguiendo un readme, que al
menos contenga las instrucciones para:
- Utilizarse creando un entorno virtual (venv)
- Instalar las dependencias necesarias con pip.
- Configurar la conexión a la base de datos.

# Configuración ✔
Las configuraciones necesarias para que el proyecto se ejecute deben poder
configurarse desde un archivo. env, .ini o similar con la librería Python-decouple.